In [23]:
import json
import re
import pandas as pd

# =========================
# CONFIG
# =========================
IN_PATH = "expekt.txt"
OUT_CSV = "expekt_player_props.csv"

COLUMNS = [
    "event",
    "player",
    "selectionLabel",
    "odds_decimal",
    "status_selection",
    "marketLabel",
    "deadline",
]

# =========================
# HELPERS
# =========================
def load_json(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        txt = f.read().strip()
        if not txt:
            raise ValueError(f"Fil er tom: {path}")
        return json.loads(txt)

def to_float(x):
    return pd.to_numeric(x, errors="coerce")

def iso_from_expekt_time(ts):
    if not isinstance(ts, str):
        return None
    s = ts.strip()
    if s.endswith("Z"):
        s = s[:-1] + "+00:00"
    return s

def normalize_event_name(name: str) -> str:
    if not isinstance(name, str):
        return None
    s = name.strip()
    s = re.sub(r"\bFC\b", "", s, flags=re.IGNORECASE)
    s = s.replace(" v ", " vs ")
    s = s.replace(" - ", " vs ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def player_name_from_outcome(name: str):
    if not isinstance(name, str):
        return None
    n = name.strip()
    if "," in n:
        last, first = [p.strip() for p in n.split(",", 1)]
        if first and last:
            return f"{first} {last}"
    return n

def selection_label_from_x_plus(x: int) -> str:
    return f"Over {x - 0.5}"

def status_to_open(s):
    s2 = str(s).strip().upper()
    if s2 in ["OPEN", "ACTIVE", "TRADING", "ONGOING"]:
        return "Open"
    return str(s).strip() if s is not None else "Open"

def outcome_is_bettable(o: dict) -> bool:
    """
    Filtrer locked odds væk.
    Typisk: isTraded == false svarer til "låst" i UI.
    Vi accepterer kun outcomes der er traded + har en åben status.
    """
    if not isinstance(o, dict):
        return False

    # Hvis Expekt siger den ikke er traded, så er den "låst"
    if o.get("isTraded") is False:
        return False

    # Ekstra sikkerhed: hvis outcome status tydeligt ikke er åben
    st = str(o.get("status", "")).strip().upper()
    if st and st not in ["OPEN", "ACTIVE", "TRADING", "ONGOING"]:
        return False

    return True

# =========================
# PARSER
# =========================
def parse_expekt_player_props(doc: dict) -> pd.DataFrame:
    markets = doc.get("markets") or []
    if not isinstance(markets, list):
        markets = []

    # Event name
    event_raw = doc.get("name")
    if not isinstance(event_raw, str) or not event_raw.strip():
        parts = doc.get("participants") or []
        if isinstance(parts, list) and len(parts) >= 2:
            a = parts[0].get("name")
            b = parts[1].get("name")
            if a and b:
                event_raw = f"{a} vs {b}"
            else:
                event_raw = "unknown_event"
        else:
            event_raw = "unknown_event"

    event_name = normalize_event_name(event_raw)
    deadline = iso_from_expekt_time(doc.get("startTime"))

    # ---------------------------------------------------------
    # Count markets (X+)
    # ---------------------------------------------------------
    count_patterns = [
        # Skud på mål
        (r"^Spiller har (\d+)\+ skud på mål$", "Antal afslutninger på mål"),

        # Skud (total)
        (r"^Spiller har (\d+)\+ skud$", "Spillers samlede antal skud"),

        # Tacklinger
        (r"^Spiller har (\d+)\+ tacklinger$", "Spillers samlede antal tacklinger"),

        # Frispark begået (fouls committed)
        (r"^Spiller begår (\d+)\+ forseelser$", "Spiller Frispark Begået"),

        # Frispark tildelt (fouls drawn / committed against player)
        # Du bad specifikt om "forseelser begået imod spiller"
        # Forseelser begået mod spiller (tallet først, som i din txt)
        (r"^(\d+)\+\s*forseelser begået mod spiller$", "Spiller Frispark Tildelt"),
        (r"^(\d+)\+\s*fouls drawn$", "Spiller Frispark Tildelt"),  # hvis du også ser engelsk

        # Assists (X+) robust: assist / assists / assistance
        (r"^Spiller laver (\d+)\+ assist$", "Spillers samlede antal assister"),
        (r"^Spiller laver (\d+)\+ assists$", "Spillers samlede antal assister"),
        (r"^Spiller laver (\d+)\+ assistance$", "Spillers samlede antal assister"),
        (r"^Spiller laver (\d+)\+ assistances$", "Spillers samlede antal assister"),

        # Offside(s) (X+)
        (r"^Spiller har (\d+)\+ offside$", "Spiller Offsides"),
        (r"^Spiller har (\d+)\+ offsides$", "Spiller Offsides"),
        (r"^Spiller har (\d+)\+ offside\(s\)$", "Spiller Offsides"),

        # Redninger
        (r"^Spiller laver (\d+)\+ redninger$", "Målmand Redninger"),
    ]

    # ---------------------------------------------------------
    # Yes/No markets
    # ---------------------------------------------------------
    yesno_patterns = [
        (r"^Scorer når som helst$", "Spiller scorer"),
        (r"^Spiller modtager et kort$", "Spiller får kort"),
        (r"^Spiller modtager et rødt kort$", "Spiller får rødt kort"),
    ]

    rows_out = []

    for m in markets:
        market_name = m.get("name")
        if not isinstance(market_name, str) or not market_name.strip():
            continue
        market_name = market_name.strip()

        status_market = status_to_open(m.get("status"))

        # Count markets
        matched = False
        for pat, mtype in count_patterns:
            mm = re.match(pat, market_name)
            if not mm:
                continue
            
            x_val = None
            for gi in range(1, len(mm.groups()) + 1):
                g = mm.group(gi)
                if g is not None and str(g).isdigit():
                    x_val = int(g)
                    break
            selection_label = selection_label_from_x_plus(x_val)

            outcomes = m.get("outcomes") or []
            if not isinstance(outcomes, list) or len(outcomes) == 0:
                matched = True
                break
            


            for o in outcomes:
                if not outcome_is_bettable(o):
                    continue
                player_raw = o.get("name")
                player = player_name_from_outcome(player_raw)
                if not player:
                    continue

                odds_dec = to_float(o.get("formatDecimal"))
                if pd.isna(odds_dec):
                    continue

                market_label = f"{mtype} | {player}"

                rows_out.append({
                    "event": event_name,
                    "player": player,
                    "selectionLabel": selection_label,
                    "odds_decimal": odds_dec,
                    "status_selection": status_market,
                    "marketLabel": market_label,
                    "deadline": deadline,
                })

            matched = True
            break

        if matched:
            continue

        # Yes/No markets
        for pat, mtype in yesno_patterns:
            if not re.match(pat, market_name):
                continue

            outcomes = m.get("outcomes") or []
            if not isinstance(outcomes, list) or len(outcomes) == 0:
                break


            for o in outcomes:
                if not outcome_is_bettable(o):
                    continue
                player_raw = o.get("name")
                player = player_name_from_outcome(player_raw)
                if not player:
                    continue

                odds_dec = to_float(o.get("formatDecimal"))
                if pd.isna(odds_dec):
                    continue

                market_label = f"{mtype} | {player}"

                rows_out.append({
                    "event": event_name,
                    "player": player,
                    "selectionLabel": "Yes",
                    "odds_decimal": odds_dec,
                    "status_selection": status_market,
                    "marketLabel": market_label,
                    "deadline": deadline,
                })
            break

    df = pd.DataFrame(rows_out, columns=COLUMNS)
    if df.empty:
        return df

    # Dedupe pr key
    df = df.sort_values(
        ["event", "marketLabel", "selectionLabel", "odds_decimal"],
        kind="stable"
    )
    df = df.drop_duplicates(
        subset=["event", "marketLabel", "selectionLabel"],
        keep="first"
    ).reset_index(drop=True)

    # Sortering
    df = df.sort_values(
        ["marketLabel", "player", "selectionLabel"],
        kind="stable"
    ).reset_index(drop=True)

    return df

def main():
    doc = load_json(IN_PATH)
    df = parse_expekt_player_props(doc)

    df.to_csv(OUT_CSV, index=False, encoding="utf-8")
    print(f"Saved {len(df)} rows to {OUT_CSV}")
    print(df.head(50))

if __name__ == "__main__":
    main()

Saved 734 rows to expekt_player_props.csv
                                       event              player  \
0   Eintracht Frankfurt vs Borussia Dortmund       Ansgar Knauff   
1   Eintracht Frankfurt vs Borussia Dortmund       Ansgar Knauff   
2   Eintracht Frankfurt vs Borussia Dortmund       Ansgar Knauff   
3   Eintracht Frankfurt vs Borussia Dortmund       Arthur Theate   
4   Eintracht Frankfurt vs Borussia Dortmund       Aurele Amenda   
5   Eintracht Frankfurt vs Borussia Dortmund        Aurelio Buta   
6   Eintracht Frankfurt vs Borussia Dortmund            Can Uzun   
7   Eintracht Frankfurt vs Borussia Dortmund            Can Uzun   
8   Eintracht Frankfurt vs Borussia Dortmund            Can Uzun   
9   Eintracht Frankfurt vs Borussia Dortmund  Carney Chukwuemeka   
10  Eintracht Frankfurt vs Borussia Dortmund  Carney Chukwuemeka   
11  Eintracht Frankfurt vs Borussia Dortmund  Carney Chukwuemeka   
12  Eintracht Frankfurt vs Borussia Dortmund     Daniel Svensson   
13  Ei

In [ ]:
    # ---------------------------------------------------------
    # Count markets (X+)
    # ---------------------------------------------------------
    count_patterns = [
        # Skud på mål
        (r"^Spiller har (\d+)\+ skud på mål$", "Antal afslutninger på mål"),

        # Skud (total)
        (r"^Spiller har (\d+)\+ skud$", "Spillers samlede antal skud"),

        # Tacklinger
        (r"^Spiller har (\d+)\+ tacklinger$", "Spillers samlede antal tacklinger"),

        # Frispark begået (fouls committed)
        (r"^Spiller begår (\d+)\+ forseelser$", "Spiller Frispark Begået"),

        # Frispark tildelt (fouls drawn / committed against player)
        # Du bad specifikt om "forseelser begået imod spiller"
        (r"^Spiller bliver udsat for (\d+)\+ forseelser$", "Spiller Frispark Tildelt"),
        (r"^Spiller bliver fældet (\d+)\+ gange$", "Spiller Frispark Tildelt"),
        (r"^Der begås (\d+)\+ forseelser imod spiller$", "Spiller Frispark Tildelt"),
        (r"^Spiller får (\d+)\+ frispark$", "Spiller Frispark Tildelt"),
        (r"^Forseelser begået mod spiller \((\d+)\+\)$", "Spiller Frispark Tildelt"),
        (r"^Forseelser begået (mod|imod) spiller\s*\((\d+)\+\)$", "Spiller Frispark Tildelt"),

        # Assists (X+) robust: assist / assists / assistance
        (r"^Spiller laver (\d+)\+ assist$", "Spillers samlede antal assister"),
        (r"^Spiller laver (\d+)\+ assists$", "Spillers samlede antal assister"),
        (r"^Spiller laver (\d+)\+ assistance$", "Spillers samlede antal assister"),
        (r"^Spiller laver (\d+)\+ assistances$", "Spillers samlede antal assister"),

        # Offside(s) (X+)
        (r"^Spiller har (\d+)\+ offside$", "Spiller Offsides"),
        (r"^Spiller har (\d+)\+ offsides$", "Spiller Offsides"),
        (r"^Spiller har (\d+)\+ offside\(s\)$", "Spiller Offsides"),

        # Redninger
        (r"^Spiller laver (\d+)\+ redninger$", "Målmand Redninger"),
    ]

    # ---------------------------------------------------------
    # Yes/No markets
    # ---------------------------------------------------------
    yesno_patterns = [
        (r"^Scorer når som helst$", "Spiller scorer"),
        (r"^Spiller modtager et kort$", "Spiller får kort"),
        (r"^Spiller modtager et rødt kort$", "Spiller får rødt kort"),
    ]